In [1]:
import numpy as np
from scipy.signal import convolve
from time import time

In [2]:
def get_random_size(random_state=None, N=1000):
    rng = np.random.RandomState(random_state)
    e = .2 + rng.rand(4) * 2
    s = (10**e).astype(np.int64)
    return (s[0], s[1]), (s[2], s[3])

def _fftconv_faster(S1, S2, mode="full", random_state=None):
    rng = np.random.RandomState(random_state)
    x = rng.randn(*S1)
    h = rng.randn(*S2)
    
    start = time()
    _ = convolve(x, h, mode=mode, method="fft")
    fft_time = time() - start
    
    start = time()
    _ = convolve(x, h, mode=mode, method="direct")
    direct_time = time() - start
    return (
        "fft" if fft_time < direct_time else "direct",
        {"fft_time": fft_time, "direct_time": direct_time},
        (x, h),
    )

In [3]:
import math
def _prod(iterable):
    """
    Product of a list of numbers.
    Faster than np.prod for short lists like array shapes.
    """
    product = 1
    for x in iterable:
        product *= x
    return product
def _predict_fftconv_faster(x, h, constant, mode="full"):
    full_out_shape = [n + k - 1 for n, k in zip(x.shape, h.shape)]
    if mode == 'full':
        out_shape = full_out_shape
    elif mode == 'same':
        out_shape = x.shape
    elif mode == 'valid':
        out_shape = [n - k + 1 for n, k in zip(x.shape, h.shape)]
    else:
        raise ValueError('mode is invalid')

    # see whether the Fourier transform convolution method or the direct
    # convolution method is faster (discussed in scikit-image PR #1792)
    direct_time = float(min(x.size, h.size) * _prod(out_shape))
    fft_time = _prod(full_out_shape) * sum(math.log(n) for n in full_out_shape)
    fastest = "fft" if direct_time - constant[0] * fft_time > constant[1] else "direct"
    times = {'pred_direct_time': direct_time, 'pred_fft_time': fft_time}
    return fastest, times
    

In [25]:
import pandas as pd
from tqdm.notebook import tqdm
constants = {"full": [0.2949277303306508, 11782.76945483908],
             "same": [0.26570413707755003, 10800.50559395122],
             "valid": [0.34633506077546283, 4483.810300140974]}
dfs = {}
for mode in ["full", "same", "valid"]:
    data = []
    constant = constants[mode]
    for seed in tqdm(range(1000)):
        S1, S2 = get_random_size(random_state=seed)
        if (mode == 'valid'
            and (S1[0] > S2[0] and S1[1] < S2[1]
                or S1[0] < S2[0] and S1[1] > S2[1])):
            S1, S2 = (S1[0], S2[1]), (S2[0], S1[1])
        fastest, times, (x, h) = _fftconv_faster(S1, S2, random_state=seed, mode=mode)
        pred_fastest, pred_times = _predict_fftconv_faster(x, h, constant)

        datum = {
            "shape1": S1,
            "shape2": S2,
            "actual_fastest": fastest,
            "predicted_fastest": pred_fastest,
            "mode": mode,
            "constant": constant,
            **times,
            **pred_times
        }
        data.append(datum)
    today = "2019-07-27"
    df = pd.DataFrame(data)
    df.to_csv(f"out/{mode}-{today}-test.csv", index=False)
    dfs[mode] = df

In [23]:
df = dfs['valid']
wrong = df[df['actual_fastest'] != df['predicted_fastest']]
wrong.head()

,shape1,shape2,actual_fastest,predicted_fastest,mode,constant,fft_time,direct_time,pred_direct_time,pred_fft_time
1,"(10, 43)","(1, 6)",direct,fft,valid,"[0.47790439279188757, 986.6281426795514]",0.000215,0.000041,2880.0,2963.417330
2,"(11, 1)","(19, 11)",direct,fft,valid,"[0.47790439279188757, 986.6281426795514]",0.000284,0.000027,3509.0,1839.095962
5,"(4, 87)","(4, 108)",direct,fft,valid,"[0.47790439279188757, 986.6281426795514]",0.000300,0.000085,472584.0,9796.297362
6,"(96, 7)","(69, 1)",direct,fft,valid,"[0.47790439279188757, 986.6281426795514]",0.000283,0.000162,79212.0,8088.551510
7,"(2, 44)","(11, 57)",direct,fft,valid,"[0.47790439279188757, 986.6281426795514]",0.000315,0.000226,105600.0,8508.092203


In [24]:
for mode, df in dfs.items():
    print(mode, (df.actual_fastest == df.predicted_fastest).sum() / len(df))

full 0.856
same 0.72
valid 0.599
